# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 17% of the assignment grade.

**Note**: Questions 2-13 rely on your question 1 answer.

In [2]:
import pandas as pd
import numpy as np

# Filter all warnings
import warnings
warnings.filterwarnings('ignore')

### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.  `'Switzerland17'` should be `'Switzerland'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [7]:
data = pd.ExcelFile("EnergyIndicators.xls")
energy = pd.read_excel(data, skiprows=17, skipfooter=38) # numbers are given

energy = energy.drop(energy.columns[1], axis=1) # excluding the columns
energy = energy.drop(energy.columns[0], axis=1)

energy.columns = ["Country", "Energy Supply", "Energy Supply per Capita", "% Renewable"] # renaming the columns

energy = energy.replace({"...": np.nan})

energy["Energy Supply"] = energy["Energy Supply"] * 1000000 # changing the joules

energy["Country"] = energy["Country"].replace({"United States of America": "United States",
                                            "United Kingdom of Great Britain and Nothern Ireland": "United Kingdom",
                                            "China, Hong Kong Special Administrative Region": "Hong Kong"})

energy['Country'] = energy["Country"].replace(r' \(.*\)', '',regex=True) # regex to be replaced
print(energy["Country"].head(30))

0                          Afghanistan
1                              Albania
2                              Algeria
3                       American Samoa
4                              Andorra
5                               Angola
6                             Anguilla
7                  Antigua and Barbuda
8                            Argentina
9                              Armenia
10                               Aruba
11                          Australia1
12                             Austria
13                          Azerbaijan
14                             Bahamas
15                             Bahrain
16                          Bangladesh
17                            Barbados
18                             Belarus
19                             Belgium
20                              Belize
21                               Benin
22                             Bermuda
23                              Bhutan
24                             Bolivia
25    Bonaire, Sint Eusta

In [9]:
GDP = pd.read_csv("world_bank.csv", skiprows=4)

GDP["Country Name"].replace({"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}, inplace=True)
print(GDP)

         Country Name Country Code                            Indicator Name  \
0               Aruba          ABW  GDP at market prices (constant 2010 US$)   
1             Andorra          AND  GDP at market prices (constant 2010 US$)   
2         Afghanistan          AFG  GDP at market prices (constant 2010 US$)   
3              Angola          AGO  GDP at market prices (constant 2010 US$)   
4             Albania          ALB  GDP at market prices (constant 2010 US$)   
..                ...          ...                                       ...   
259       Yemen, Rep.          YEM  GDP at market prices (constant 2010 US$)   
260      South Africa          ZAF  GDP at market prices (constant 2010 US$)   
261  Congo, Dem. Rep.          COD  GDP at market prices (constant 2010 US$)   
262            Zambia          ZMB  GDP at market prices (constant 2010 US$)   
263          Zimbabwe          ZWE  GDP at market prices (constant 2010 US$)   

     Indicator Code          1960      

In [10]:
ScimEn = pd.read_excel("scimagojr-3.xlsx")

In [11]:
GDP.rename(columns={"Country Name": "Country"}, inplace=True)
GDP = GDP[['Country', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

initial_merge = pd.merge(ScimEn, energy, how="inner", left_on="Country", right_on="Country")
final_merge = pd.merge(initial_merge, GDP, how="inner", left_on="Country", right_on="Country")

final_merge.set_index("Rank", inplace=True)
sorted_merge = final_merge.sort_index()

reduced_merge = sorted_merge.head(15)

reduced_merge # my pc had some problems to process the scimagojr-3 file

,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Rank,,,,,,,,,,,,,,,,,,,,
5,Russian Federation,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
6,Canada,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
7,Germany,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
8,India,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
13,Iran,8896,8819,57470,19125,6.46,72,9.172000e+09,119.0,5.707721,3.895523e+11,4.250646e+11,4.289909e+11,4.389208e+11,4.677902e+11,4.853309e+11,4.532569e+11,4.445926e+11,4.639027e+11,NaN
15,Brazil,8668,8596,60702,14396,7.00,86,1.214900e+10,59.0,69.648030,1.845080e+12,1.957118e+12,2.056809e+12,2.054215e+12,2.208872e+12,2.295245e+12,2.339209e+12,2.409740e+12,2.412231e+12,2.319423e+12
17,Turkey,5879,5827,91857,23165,15.62,104,4.897000e+09,64.0,28.536690,6.680034e+11,6.991896e+11,7.037962e+11,6.698319e+11,7.311681e+11,7.953116e+11,8.122315e+11,8.462844e+11,8.718432e+11,9.065849e+11
18,Norway,5675,5634,32693,6024,5.76,74,1.388000e+09,273.0,97.635580,4.190512e+11,4.313284e+11,4.329860e+11,4.259607e+11,4.285247e+11,4.326762e+11,4.445694e+11,4.490094e+11,4.589539e+11,4.662883e+11
20,Sweden,4565,4494,57889,9632,12.68,95,2.044000e+09,212.0,46.598460,4.726099e+11,4.887020e+11,4.859797e+11,4.607834e+11,4.883793e+11,5.013917e+11,4.999562e+11,5.061616e+11,5.176362e+11,5.388264e+11


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [17]:
print(len(sorted_merge) - len(reduced_merge))

126


### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [12]:
GDP = pd.read_csv("world_bank.csv", skiprows=4)

GDP["Country Name"].replace({"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}, inplace=True)
GDP.rename(columns={"Country Name": "Country"}, inplace=True)

year_columns = [str(i) for i in range(1960, 2016)]
GDP["avgGDP"] = GDP[year_columns].mean(axis=1, skipna=True)

avgGDP = pd.Series(GDP["avgGDP"])
avgGDP.index = GDP["Country"]

avgGDP.sort_values(inplace=True, ascending=False)
avgGDP.head(15)

# these are the top 15 regions when it comes to GDP
# if I were to put only countries, it would take a lot of desnecessary amount of time while I can show my understandings with this

Country
World                        3.739578e+13
High income                  2.783321e+13
OECD members                 2.763563e+13
Post-demographic dividend    2.652996e+13
Europe & Central Asia        1.522987e+13
European Union               1.113840e+13
North America                1.004493e+13
IDA & IBRD total             9.886159e+12
Low & middle income          9.238962e+12
IBRD only                    9.112065e+12
Middle income                9.083337e+12
United States                9.051161e+12
Euro area                    8.370114e+12
East Asia & Pacific          7.882309e+12
Upper middle income          7.003400e+12
Name: avgGDP, dtype: float64

### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [13]:
# As seen in the last question, the country in this question is the EU

GDP.reset_index(inplace=True)
value_2015 = GDP.loc[GDP["Country"] == "European Union", "2015"]
value_2005 = GDP.loc[GDP["Country"] == "European Union", "2005"]

print(value_2015 - value_2005) 

70    1.561817e+12
dtype: float64


### Question 5
What is the mean energy supply per capita?

*This function should return a single number.*

In [97]:
print(energy["Energy Supply per Capita"].mean(skipna=True))

90.66666666666667


### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [14]:
energy.set_index("Country", inplace=True)
maximum_energy = max(energy["Energy Supply per Capita"])
tup = (energy[energy["Energy Supply per Capita"] == maximum_energy].index[0], maximum_energy)
print(tup)

('Iceland', 957.0)


### Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [4]:
ScimEn["Ratio Self-Citations"] = ScimEn["Self-citations"] / ScimEn["Citations"]
ScimEn.set_index("Country", inplace=True)

max_ratio = max(ScimEn["Ratio Self-Citations"])
tup = (ScimEn[ScimEn["Ratio Self-Citations"] == max_ratio].index[0], max_ratio)
print(tup)

('French Guiana', 1.0)


### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [25]:
energy["Population"] = energy["Energy Supply"] / energy["Energy Supply per Capita"]
energy.sort_values(by="Population", ascending=False, inplace=True)
print(energy.iloc[2].name)

United States of America20


### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).


*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [30]:
sorted_merge["Population"] = sorted_merge["Energy Supply"] / sorted_merge["Energy Supply per Capita"]
sorted_merge["Citable Documents per Person"] = sorted_merge["Citable documents"] / sorted_merge["Population"]

per_capita_df = sorted_merge[["Citable Documents per Person", "Energy Supply per Capita"]]

correlation_matrix = per_capita_df.corr()
correlation_matrix

,Citable Documents per Person,Energy Supply per Capita
Citable Documents per Person,1.000000,0.537729
Energy Supply per Capita,0.537729,1.000000


### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [48]:
top_15 = sorted_merge.head(15)
median = top_15["% Renewable"].median()

sorted_merge["HighRenew"] = 0
sorted_merge.loc[sorted_merge["% Renewable"] >= median, "HighRenew"] = 1

HighRenew = pd.Series(sorted_merge["HighRenew"])
HighRenew.index = sorted_merge["Country"]

HighRenew

Country
Russian Federation    1
Canada                1
Germany               1
India                 0
Iran                  0
                     ..
Maldives              0
Belize                1
Palau                 1
Guyana                0
Mauritania            0
Name: HighRenew, Length: 141, dtype: int64